In [18]:
import cv2
#CV2 loads everythings as BGR
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label
from tqdm import tqdm

test_image_list = glob.glob("./test_images/*.jpg")

In [ ]:
#HOG Feature Extractor
def patch_analyzer(image, size=(16,16), nbins=16, bins_range=(0,256)):
    grey_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    fd, hog_image = hog(grey_img, orientations=8, pixels_per_cell=(8, 8),
                        cells_per_block=(1, 1), visualise=True)

    #Color Space Extraction

    #conv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    conv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)

    #Downsample 0.25x from original
    #Original patch is 64px
    # Use cv2.resize().ravel() to create the feature vector
    scale_feature = cv2.resize(conv_img, size).ravel() 


    #Create Histograms from scaled image
    c1_hist = np.histogram(image[:,:,0], bins=nbins, range=bins_range)
    c2_hist = np.histogram(image[:,:,1], bins=nbins, range=bins_range)
    c3_hist = np.histogram(image[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((c1_hist[0], c2_hist[0], c3_hist[0]))


    #Combine and Normalized Data
    feature_list = [fd, scale_feature, hist_features]
    # Create an array stack, NOTE: StandardScaler() expects np.float64
    tmp_x = np.concatenate(feature_list).astype(np.float64)

    return tmp_x, image, hog_image

In [ ]:
#Train the classifier using patch_analyzer
vehicles = glob.glob('vehicles/*/*.png')
non_vehicles = glob.glob('non-vehicles/*/*.png')
image_loc_list = non_vehicles + vehicles

features_array = np.asarray([patch_analyzer(cv2.imread(image))[0] for image in tqdm(image_loc_list)])

np.save('LinearSVC_GTI_KTTT', features_array)


In [29]:
#Number of non-cars are represented by zeros
#Number of cars are represented by ones
img_labels = np.concatenate([np.zeros(8968), np.ones(8792)])

features_array = np.load('LinearSVC_GTI_KTTT.npy')

X_train, X_test, y_train, y_test= train_test_split(features_array, img_labels, test_size=0.35)

# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)

print("LinearSVC Reached")

#Implement Linear SVM Classifier
clf = LinearSVC(C=0.001,verbose=1, random_state=0)
clf.fit(scaled_X_train, y_train)

#Run trained classifier on test image
print()

print(clf.score(scaled_X_train, y_train)*100)

print(clf.score(scaled_X_test, y_test)*100)

LinearSVC Reached
[LibLinear]
99.5235620236
98.2786357786


In [15]:
X_train = np.load('LinearSVC_GTI_KTTT.npy')
y_train = np.concatenate([np.zeros(8968), np.ones(8792)])

# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
scaled_X_train = X_scaler.transform(X_train)

print("LinearSVC Reached")

#Implement Linear SVM Classifier
clf = LinearSVC(C=0.001,verbose=1, random_state=0)
clf.fit(scaled_X_train, y_train)

LinearSVC Reached
[LibLinear]

LinearSVC(C=0.001, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=1)

In [39]:
#Return a list of image patches
test_image = cv2.imread(test_image_list[0])

def sliding_window(img, y_start=0, y_stop=1280, patch_size=64, pix_per_cell=8, cells_per_block=2, cells_per_step=1, orient=8, scale=1):
    draw_img = np.copy(img)
    #Convert to float-32 value and normalize
    image = img.astype(np.float32)/255
    img_window = img[y_start:y_stop,:,:]
    trans_img = cv2.cvtColor(img_window, cv2.COLOR_BGR2HLS)
    #Uncertain why they convert to YCC here
    #trans_img = convert_color(img_tosearch, conv='RGB2YCrCb')
    if scale != 1:
        imshape = trans_img.shape
        trans_img = cv2.resize(trans_img, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = trans_img[:,:,0]
    ch2 = trans_img[:,:,1]
    ch3 = trans_img[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell)//cells_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell)//cells_per_block + 1 

    nxsteps = nxblocks // cells_per_step
    nysteps = nyblocks // cells_per_step

    patch_list = []
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*pix_per_cell*cells_per_step
            xpos = xb*cells_per_step
            ypos_end = ypos+patch_size
            xpos_end = xpos+patch_size
            cur_patch = trans_img[ypos:ypos_end, xpos:xpos_end]
            # Extract HOG for this patch
            patch_list.append([cur_patch, (xpos, ypos), (xpos_end, ypos_end)])
    return patch_list

#Iterate through three different scales
scale_list = [0.5, 1, 1.5]

for scale in scale_list:
    sliding_window(test_image, y_stop = 500)
    print(scale)

0.5
1
1.5


In [ ]:
#Feed function trained LinearSVC classifier and list of patches to predict locations
def search_windows(img_list):
	positives_list = []
	for entry in img_list:
        prediction = clf.predict(X_scaler.transform(patch_analyzer(entry[0])[0]))
        if prediction == 1:
            positives_list.append(entry[1:])
    return positives_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy


#Overlay outputs into heatmap, generate heatmap image + bounding box cluster images

def heat_map(img, box_list, threshold=0):
	heat = np.zeros_like(image[:,:,0]).astype(np.float)
	#Iterate through all the positively identified windows
	for entry in box_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
		img[entry[0][1]:entry[1][1], entry[0][0]:box[1][0]] += 1
	#Zero out pixels below the threshold
	img[img <= threshold] = 0
	return img

#Labeled box code same as lesson function
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

#Draw bounding boxes around potential cars
draw_image = np.copy(test_image)

#Generate a list of boxes around positively identified cars
box_list = search_windows(draw_image)  

# Add heat to each box in box list and threshold the image for false positives
heat = heat_map(heat,box_list, 2)

# Visualize the heatmap when displaying    
heatmap = np.clip(heat, 0, 255)

# Find final boxes from heatmap using label function
labels = label(heatmap)
draw_img = draw_labeled_bboxes(draw_image, labels)

fig = plt.figure()
plt.subplot(121)
plt.imshow(draw_img)
plt.title('Car Positions')
plt.subplot(122)
plt.imshow(heatmap, cmap='hot')
plt.title('Heat Map')
fig.tight_layout()